In [1]:
import os
import pandas as pd

df = pd.read_excel('subject_info\\Brain age_PSG_raw_Total_201216(whole_data).xlsx')#, index_col=0)#, encoding='ISO-8859-1')

# DataFrame initial setting
df = df.iloc[2:, :]
df.columns = df.iloc[0,:]
df.drop(index=[2], inplace=True)
df['age'] = df['age'].copy()*(-1)
df_origin = df.copy()

# AHI == '.'인 데이터 삭제
Ex_dot_subjects = df_origin.loc[df_origin['AHI - Total index A+H'] == '.', 'PSG study Number#']
df = df[df['AHI - Total index A+H'] != '.'].copy()

# AHI == 'NaN'인 데이터 삭제
Ex_AHI_nan_subjects = df_origin.loc[df_origin['AHI - Total index A+H'].isna(), ['PSG study Number#', 'AHI - Total index A+H']]
df.dropna(subset=['AHI - Total index A+H'], inplace=True)

# ISI == 'NaN'인 데이터 삭제
Ex_ISI_nan_subjects = df_origin.loc[df_origin['ISI  Total-2'].isna(), ['PSG study Number#', 'ISI  Total-2']]
df.dropna(subset=['ISI  Total-2'], inplace=True)

# age가 20세 미만인 subject 
ExAge_subject = df_origin.loc[df_origin['age']<20, 'PSG study Number#']

# (TIB-SOL)가 330 미만인 subject 
df_TIB = df_origin['TIB - Total Recording time (min)'].astype(float)
df_SOL = df_origin['Sleep latency (min)'].astype(float)
ExSOL_subject = df_origin.loc[(df_TIB-df_SOL)<330, 'PSG study Number#']

# 20세 이상 and period after SOL 330 이상인 데이터만 포함
df = df[ (df['age']>=20) & ((df_TIB-df_SOL)>=330)]

print('Excluded because of nan AHI value --> n={}'.format(Ex_AHI_nan_subjects.size))
print('Excluded because of dot AHI value --> n={}'.format(Ex_dot_subjects.size))
print('Excluded because of nan ISI value --> n={}'.format(Ex_ISI_nan_subjects.size))
print('Excluded because of age under 20 --> n={}'.format(ExAge_subject.size))
print('Excluded because of period shorter than 330 m --> n={}'.format(ExSOL_subject.size))

df

Excluded because of nan AHI value --> n=38
Excluded because of dot AHI value --> n=3
Excluded because of nan ISI value --> n=406
Excluded because of age under 20 --> n=216
Excluded because of period shorter than 330 m --> n=229


C:\Users\CYS\AppData\Local\Temp\ipykernel_18972\1577895506.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[ (df['age']>=20) & ((df_TIB-df_SOL)>=330)]


2,환자번호,이름,생년월일,성별,age,PSG-RA DATA 검사일,PSG study Number#,height,weight,BMI,...,Component 5-2,Component 5-3,Component 6,Component 7-3,Component 7-4,NaN,NaN,수정하지 말 것!!!,NaN,함수 숨기기 되어있음
3,35530991,현매선,1944-06-19,F,71.669444,2016-02-20,PE160128,130,48,28.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,77935,이강환,1960-06-18,M,57.461111,2017-12-04 00:00:00,LE001,178,82,25.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,39113664,박준,1988-08-12,M,29.319444,2017-12-07 00:00:00,LE002,177,72,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,36227186,한정윤,1990-06-12,F,27.541667,2017-12-27 00:00:00,LE003,160,55,21.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,37930979,김채은,1997-04-25,F,20.730556,2018-01-18 00:00:00,LE006,157,61,24.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5280,38563253,조현상,1969-12-03,M,47.813889,2017-09-26 00:00:00,ST17027,172,85,28.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5281,18959296,김성민,1979-06-26,M,38.258333,2017-09-29 00:00:00,ST17028,172,76,25.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5282,38619628,조성환,1987-01-19,M,30.725,2017-10-10 00:00:00,ST17029,174,74,24.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5283,38776428,임대형,1988-10-03,M,29.027778,2017-10-13 00:00:00,ST17030,170,100,34.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# df에 'class' column을 추가
df.loc[:, 'class'] = 0
df_AHI = df['AHI - Total index A+H'].astype(float)
df_ISI = df['ISI  Total-2']

# 각 range 조건에 따라서, 해당하는 class를 'class' column에 대입
# 1. Healthy: AHI < 15 and ISI < 15
con_AHI = df_AHI < 15
con_ISI = df_ISI < 15
con1 = con_AHI & con_ISI

# 2. OSA: AHI >= 15 and ISI < 15
con_AHI = df_AHI >= 15
con_ISI = df_ISI < 15
con2 = con_AHI & con_ISI

# 3. INS: AHI < 15 and ISI >= 15
con_AHI = df_AHI < 15
con_ISI = df_ISI >= 15
con3 = con_AHI & con_ISI

# 4. COMISA: AHI >= 15 and ISI >=15
con_AHI = df_AHI >= 15
con_ISI = df_ISI >= 15
con4 = con_AHI & con_ISI

df.loc[con1, 'class'] = 'Healthy'
df.loc[con2, 'class'] = 'OSA'
df.loc[con3, 'class'] = 'Insomnia'
df.loc[con4, 'class'] = 'COMISA'

df['class']

# class가 부여되지 않은 subject가 있는지 확인
print('Case: Not assigned with class --> n={}'.format(df[['class', '이름']][(df['class'] != 'Healthy') & (df['class'] != 'OSA') & (df['class'] != 'Insomnia') & (df['class'] != 'COMISA')].size))

Case: Not assigned with class --> n=0


In [3]:
# class column에 따라서 각 class 추출
subject_healthy = df.loc[df['class']=='Healthy', 'PSG study Number#']
subject_OSA = df.loc[df['class']=='OSA', 'PSG study Number#']
subject_Insomnia = df.loc[df['class']=='Insomnia', 'PSG study Number#']
subject_COMISA = df.loc[df['class']=='COMISA', 'PSG study Number#']
print('Healthy:{}, OSA:{}, INS:{}, COMISA:{}'.format(len(subject_healthy), len(subject_OSA), len(subject_Insomnia), len(subject_COMISA)))

df_total_subject = pd.DataFrame(columns = ['Healthy(n=1283)', 'OSA(n=1933)', 'Insomnia(n=727)', 'COMISA(n=785)'])
columns = df_total_subject.columns

df_total_subject[columns[1]] = subject_OSA.values
df_total_subject.loc[0:len(subject_healthy)-1, columns[0]] = subject_healthy.values
df_total_subject.loc[0:len(subject_Insomnia)-1, columns[2]] = subject_Insomnia.values
df_total_subject.loc[0:len(subject_COMISA)-1, columns[3]] = subject_COMISA.values

df_total_subject.to_csv('subject_info\\PSG_list_by_CYS.csv', index=False)
df_total_subject

Healthy:1283, OSA:1933, INS:727, COMISA:785


,Healthy(n=1283),OSA(n=1933),Insomnia(n=727),COMISA(n=785)
0,LE001,LE026,PE160128,PE141014
1,LE002,LE036,N15003,PE141019
2,LE003,N15006,N15009,PE141020
3,LE006,N15007,PE141015,PE141038
4,LE007,PE141017,PE141022,PE141039
...,...,...,...,...
1928,NaN,ST17015,NaN,NaN
1929,NaN,ST17019,NaN,NaN
1930,NaN,ST17027,NaN,NaN
1931,NaN,ST17030,NaN,NaN


In [4]:
new_df = df[['PSG study Number#', 'class', 'Sleep latency (min)', 'TIB - Total Recording time (min)']]
new_df.to_csv('subject_info\\all_class.csv', index=False)
new_df

2,PSG study Number#,class,Sleep latency (min),TIB - Total Recording time (min)
3,PE160128,Insomnia,2.0,476.0
4,LE001,Healthy,5,373.5
5,LE002,Healthy,3,351
6,LE003,Healthy,2.5,365.4
8,LE006,Healthy,1.5,387.5
...,...,...,...,...
5280,ST17027,OSA,2,398
5281,ST17028,Healthy,10.5,390.5
5282,ST17029,Healthy,5.5,361.5
5283,ST17030,OSA,5,354.5


In [15]:
sum(new_df['PSG study Number#'] == 'PE190265')

1

In [ ]:
# overlap subject test
overlaped = subject_COMISA.value_counts()
overlaped[overlaped>1]